In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import math
import numpy as np
import pandas as pd
import warnings

import matplotlib.pyplot as plt
import seaborn as sns

import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold, cross_val_score, train_test_split

import tensorflow_probability as tfp
from keras.models import Sequential
from keras.layers import Dense, LSTM

from tensorflow.keras.layers import StringLookup
import joblib
from tqdm import tqdm

#ignore warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
# brewer2mpl.get_map args: set name set type number of colors
import matplotlib as mpl

#mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=colors)

plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.dpi'] = 300

colors = ['#1E90FF', '#f12c60', '#fff21e', '#1efff2', '#1eafff', 'blueviolet', 'sandybrown', 'magenta', 'slategray']

Profile construction

In [ ]:
df_res = pd.read_csv('df_res.csv')

wind_array = df_res.to_numpy()

In [ ]:
# window lenght is m
m = 1000

nrm = [[] for j in range(len(wind_array) // m - 1 )]

for i in range(0, len(wind_array) // m - 1):
    nrm[i] = wind_array[i*(m):i*(m) + m]

In [ ]:
for i in range(342, len(nrm)):
  data = nrm[i]
  fig, axes = plt.subplots(3, 5, figsize=(25, 10), dpi=300, sharex=True, sharey=True)
  m, n = 0, 0
  for j in range(14):
    axes[m][n].scatter(data[:,2*j], data[:,2*j+1] , s=3, c='darkgray',)
    n += 1
    if n > 4:
      m += 1
      n = 0
  plt.savefig('img/%s.jpg'%i)

# Degradation check of 7-th wind turbine: 382, 391

In [ ]:
# for _1048; j=6
degradation_index = [368, 369, 370, 375, 376, 377, 379, 382, 383, 384, 385, 386, 387, 389, 390,]

# for _1043; j=1
degradation_index2 = [364, 365, 366, 367, 368, 369, 370, 375, 376, 383, 384, 385, 386, 389,]

In [ ]:
i = 368
data = nrm[i]
plt.figure(figsize=(5,3))

for j in range(14):
  if j !=1 and j !=6:
    if j ==13:
      plt.scatter(data[:,2*j], data[:,2*j+1], c='darkgray', s=2, label='Other WT')
    else:
      plt.scatter(data[:,2*j], data[:,2*j+1], c='darkgray', s=2,)

j = 1
plt.scatter(data[:,2*j], data[:,2*j+1], c='crimson', s=2, label='WT #2')
plt.xlim(1,16)
plt.xlabel('Wind Speed (m/s)', fontsize=10)
plt.ylabel('Wind Power (kW)', fontsize=10)
plt.legend(fontsize=10)

## Relative plot for illustration

covariate matching

In [ ]:
nrm0 = nrm[368]
speed0 = nrm0[:, 0::2]
power0 = nrm0[:, 1::2]

In [ ]:
def find_nearest(a, a0):
    "Element in nd array `a` closest to the scalar value `a0`"
    idx = np.abs(a - a0).argmin()
    return idx

In [ ]:
fig, axes = plt.subplots(figsize=(5, 3), dpi=300)

sns.kdeplot(speed0[:,0], label='WT #1', fill=False,)
sns.kdeplot(speed0[:,1], label='WT #2', fill=False,linestyle='--')
sns.kdeplot(speed0[:,12], label='WT #13', fill=False,linestyle='--')

axes.legend(prop={'size': 12},loc='best')

axes.tick_params(axis='x', labelsize=16)
axes.tick_params(axis='y', labelsize=16)
axes.set_xlabel('Wind Speed (m/s)',fontsize=16)
axes.set_ylabel('Density',fontsize=16)
axes.set_title('Before Matching')

In [ ]:
omega = 0.2
std = np.std(speed0[:,0])
speed0_new = [[] for _ in range(14)]
power0_new = [[] for _ in range(14)]
speed0_new[0] = speed0[:,0]
power0_new[0] = power0[:,0]

for j in range(1, 14):
  for i in range(len(speed0[:,0])):
    idx = find_nearest(speed0[:,j], speed0[:,0][i])
    tmp = speed0[:,j].flat[idx]
    if abs(tmp - speed0[:,0][i]) <  omega*std:
      speed0_new[j].append(tmp)
      power0_new[j].append(power0[:,j].flat[idx])

fig, axes = plt.subplots(figsize=(5, 3), dpi=300)

sns.kdeplot(speed0[:,0], label='WT #1', fill=False,)
sns.kdeplot(speed0_new[1], label='WT #2', fill=False,linestyle='--')
sns.kdeplot(speed0_new[12], label='WT #13', fill=False,linestyle='--')

axes.legend(prop={'size': 12},loc='best')

axes.tick_params(axis='x', labelsize=16)
axes.tick_params(axis='y', labelsize=16)
axes.set_xlabel('Wind Speed (m/s)',fontsize=16)
axes.set_ylabel('Density',fontsize=16)
axes.set_title('After Matching')

In [ ]:
sns.kdeplot(speed0[:,0], label='WT #1', fill=False,)
sns.kdeplot(speed0_new[1], label='WT #2', fill=False,linestyle='--')
sns.kdeplot(speed0_new[12], label='WT #13', fill=False,linestyle='--')

In [ ]:
plt.figure(figsize=(5,3))

plt.scatter(speed0_new[0], power0_new[0], c='darkgray', s=2,label='WT #1')
plt.scatter(speed0_new[12], power0_new[12], c='dodgerblue', s=2, label='WT #13')
plt.scatter(speed0_new[1], power0_new[1], c='crimson', s=2, label='WT #2')
plt.xlim(1,16)
plt.xlabel('Wind Speed (m/s)', fontsize=10)
plt.ylabel('Wind Power (kW)', fontsize=10)
plt.legend(fontsize=10)

QPC

In [ ]:
import os
import numpy as np
import pandas as pd
import time
import math
import tensorflow as tf
from keras import Model
from keras.models import Sequential
import keras.backend as K
from keras.layers import *
from keras.models import load_model

from keras import regularizers
from keras.constraints import non_neg
from keras.initializers import RandomUniform,RandomNormal
from matplotlib import pyplot as plt

tfd = tfp.distributions

In [ ]:
def qloss(y_true, y_pred, n_q=99):
    q = np.array(range(1, n_q + 1))
    left = (q / (n_q + 1) - 1) * (y_true - y_pred)
    right = q / (n_q + 1) * (y_true - y_pred)
    return K.mean(K.maximum(left, right))

from keras.callbacks import *

def QRNN(num_quantiles):
  mean_inputs = tf.keras.Input(16)
  mean_x = Dense(256,)(mean_inputs)
  mean_x=Dense(256,)(mean_x)
  mean_x = Dense(128,)(mean_x)
  mean_x = Dense(64,)(mean_x)
  mean_x = Dense(32,)(mean_x)
  outputs=Dense(num_quantiles)(mean_x)
  model = keras.Model(inputs=mean_inputs, outputs=outputs)
  return model

def QRNN_2(num_quantiles):
  mean_inputs = tf.keras.Input((1,1))
  mean_x = LSTM(64, return_sequences=True,)(mean_inputs)
  mean_x=LSTM(32, return_sequences=False)(mean_x)
  mean_x = Dense(16,)(mean_x)
  mean_x = Dense(8,)(mean_x)

  outputs = Dense(num_quantiles)(mean_x)
  model = keras.Model(inputs=mean_inputs, outputs=outputs)
  return model

model

In [ ]:
for m in range(188, len(nrm)):
  nrm0 = nrm[m]
  speed0 = nrm0[:, 0::2]
  power0 = nrm0[:, 1::2]

  omega = 0.2
  std = np.std(speed0[:,0])
  speed0_new = [[] for _ in range(14)]
  power0_new = [[] for _ in range(14)]
  speed0_new[0] = speed0[:,0]
  power0_new[0] = power0[:,0]

  for j in range(1, 14):
    for i in range(len(speed0[:,0])):
      idx = find_nearest(speed0[:,j], speed0[:,0][i])
      tmp = speed0[:,j].flat[idx]
      if abs(tmp - speed0[:,0][i]) <  omega*std:
        speed0_new[j].append(tmp)
        power0_new[j].append(power0[:,j].flat[idx])

  for k in range(14):
    speed_list = np.array(speed0_new[k])
    speed_list = speed_list.reshape(len(speed_list), 1)
    power_list = np.array(power0_new[k])/2000

    num_quantiles = 9
    model = QRNN_2(num_quantiles)

    # Train
    model.compile(loss=lambda y_t, y_p: qloss(y_true=y_t, y_pred=y_p, n_q=num_quantiles),
                  optimizer='adam')

    model.fit(speed_list, power_list, batch_size = 64, epochs = 200, validation_split = 0.15)

    model_save_path = "models/Nw1000_QPC/quantile_%s_%s.h5"%(m,k)
    # 保存模型权重
    model.save_weights(model_save_path)

plot

In [ ]:
nrm0 = nrm[368]
speed0 = nrm0[:, 0::2]
power0 = nrm0[:, 1::2]

omega = 0.2
std = np.std(speed0[:,0])
speed0_new = [[] for _ in range(14)]
power0_new = [[] for _ in range(14)]
speed0_new[0] = speed0[:,0]
power0_new[0] = power0[:,0]

for j in range(1, 14):
  for i in range(len(speed0[:,0])):
    idx = find_nearest(speed0[:,j], speed0[:,0][i])
    tmp = speed0[:,j].flat[idx]
    if abs(tmp - speed0[:,0][i]) <  omega*std:
      speed0_new[j].append(tmp)
      power0_new[j].append(power0[:,j].flat[idx])

m, n = 0, 0
fig, axes = plt.subplots(4, 4, figsize=(25, 20), dpi=300, sharex=True, sharey=True)

colors = ['#1efff2', '#f12c60', 'magenta', '#1efff2', '#1eafff', 'blueviolet', 'sandybrown', 'magenta', '#f12c60']

for k in range(14):
  print("Current segment: %s; WT: %s"%(i, k))
  speed_list = np.array(speed0_new[k])
  speed_list = speed_list.reshape(len(speed_list), 1)
  power_list = np.array(power0_new[k])

  power_list = (power_list - 0)/2000 # 设置在0-1之间
  power_list[power_list>1] = 1
  power_list[power_list<0] = 0

  num_quantiles = 9
  model = QRNN_2(num_quantiles)

  # Train
  model.load_weights("models4plot/quantile_%s.h5"%k)

  test_pred = model.predict(speed_list)

  axes[m][n].scatter(speed_list, power_list , s=7, c='darkgray', label = 'Actual data')

  for d in range(0, num_quantiles, 2):
    preds = test_pred[:, d]
    preds[preds < 0] = 0
    preds[preds > 1] = 1
    value = (d + 1)*10
    axes[m][n].scatter(speed_list, preds, c=colors[d], label = "Quantile level =  %s%%"%str(value), s=1)

  axes[m][n].set_title('WT #%s'%(k+1), fontsize=17)
  axes[m][n].set_xlim(2,16)
  axes[m][n].set_ylim(-0.05,1.05)
  axes[m][n].tick_params(labelsize=17)
  if n==0:
    axes[m][n].set_ylabel('Wind Power', fontsize=17)

  if m==3:
    axes[m][n].set_xlabel('Wind Speed', fontsize=17)

  n += 1
  if n >3:
    m += 1
    n = 0

axes[3][2].set_xlabel('Wind Speed', fontsize=17)
axes[3][2].tick_params(labelsize=17)
axes[3][3].set_xlabel('Wind Speed', fontsize=17)
axes[3][3].tick_params(labelsize=17)
lgd = axes[0][0].legend(bbox_to_anchor=(0, 1.2), loc=3, borderaxespad=0.1, ncol=4, fontsize=20)
for legend_handle in lgd.legendHandles:
    legend_handle.set_sizes([30])

DPC

In [ ]:
def BNN_normal():
  mean_inputs = tf.keras.Input((1,1))
  mean_x = LSTM(64, return_sequences=True,)(mean_inputs)
  mean_x=LSTM(32, return_sequences=False)(mean_x)
  mean_x = Dense(16,)(mean_x)
  mean_x = Dense(8,)(mean_x)

  mean_x = Dense(2)(mean_x)
  outputs=tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t[..., 0],scale=1e-3+tf.math.softplus(0.05 * t[..., 1])))(mean_x)
  # outputs = Dense(1)(mean_x)
  model = keras.Model(inputs=[mean_inputs], outputs=[outputs])
  return model

def BNN_gamma1():
  mean_inputs = tf.keras.Input((1,1))
  mean_x = LSTM(32, return_sequences=True,)(mean_inputs)
  mean_x=LSTM(16, return_sequences=False)(mean_x)
  mean_x = Dense(16,)(mean_x)
  mean_x = Dense(8,)(mean_x)

  outputs=Dense(1)(mean_x)
  model = keras.Model(inputs=mean_inputs,outputs=outputs)
  return model

def BNN_gamma2():
  mean_inputs = tf.keras.Input((1,1))
  mean_x = LSTM(32, return_sequences=True,)(mean_inputs)
  mean_x=LSTM(16, return_sequences=False)(mean_x)
  mean_x = Dense(16,)(mean_x)
  mean_x = Dense(8,)(mean_x)

  mean_x=Dense(2)(mean_x)
  outputs=tfp.layers.DistributionLambda(lambda t: tfd.Gamma(concentration=1e-3+tf.abs(t[..., 0]),rate=1e-3+tf.abs(t[..., 1])))(mean_x)
  model = keras.Model(inputs=mean_inputs,outputs=outputs)
  return model

import keras.backend as K
import math as m
def negative_loglikelihood(targets,estimated_distribution):
  return -estimated_distribution.log_prob(targets)

model

In [ ]:
power_list

In [ ]:
nrm0 = nrm[368]
speed0 = nrm0[:, 0::2]
power0 = nrm0[:, 1::2]

# omega = 0.2
# std = np.std(speed0[:,0])
# speed0_new = [[] for _ in range(14)]
# power0_new = [[] for _ in range(14)]
# speed0_new[0] = speed0[:,0]
# power0_new[0] = power0[:,0]

# for j in range(1, 14):
#   for i in range(len(speed0[:,0])):
#     idx = find_nearest(speed0[:,j], speed0[:,0][i])
#     tmp = speed0[:,j].flat[idx]
#     if abs(tmp - speed0[:,0][i]) <  omega*std:
#       speed0_new[j].append(tmp)
#       power0_new[j].append(power0[:,j].flat[idx])

for k in range(13, 14):
  speed_list = np.array(np.array(speed0)[:,k])
  speed_list = speed_list.reshape(len(speed_list), 1)
  power_list = np.array(power0)[:,k]/2000
  power_list[power_list<=0]=0.01
  model = BNN_gamma2()

  # Train
  learning_rate = 0.01
  # model.compile(optimizer='adam',loss='mse',metrics=[keras.metrics.RootMeanSquaredError()],)
  model.compile(optimizer='adam',loss=negative_loglikelihood,metrics=[keras.metrics.RootMeanSquaredError()],)
  loss_list = []
  model_list = []
  history = model.fit(speed_list, power_list, batch_size=128, epochs=995, validation_split=0)

  history = model.fit(speed_list, power_list, batch_size=64, epochs=1, validation_split=0)
  loss_list.append(history.history['loss'][0])
  model_list.append(model)
  history = model.fit(speed_list, power_list, batch_size=64, epochs=1, validation_split=0)
  loss_list.append(history.history['loss'][0])
  model_list.append(model)
  history = model.fit(speed_list, power_list, batch_size=64, epochs=1, validation_split=0)
  loss_list.append(history.history['loss'][0])
  model_list.append(model)
  history = model.fit(speed_list, power_list, batch_size=64, epochs=1, validation_split=0)
  loss_list.append(history.history['loss'][0])
  model_list.append(model)
  history = model.fit(speed_list, power_list, batch_size=64, epochs=1, validation_split=0)
  loss_list.append(history.history['loss'][0])
  model_list.append(model)
  ind = np.argmin(loss_list)
  res_model = model_list[ind]
  model_save_path = "models4plot/density_%s.h5"%k
  # 保存模型权重
  res_model.save_weights(model_save_path)

plot

In [ ]:
from scipy.stats import norm, gamma
import matplotlib.colors as mcolors
import random

nrm0 = nrm[368]
speed0 = nrm0[:, 0::2]
power0 = nrm0[:, 1::2]

omega = 0.2
std = np.std(speed0[:,0])
speed0_new = [[] for _ in range(14)]
power0_new = [[] for _ in range(14)]
speed0_new[0] = speed0[:,0]
power0_new[0] = power0[:,0]

for j in range(1, 14):
  for i in range(len(speed0[:,0])):
    idx = find_nearest(speed0[:,j], speed0[:,0][i])
    tmp = speed0[:,j].flat[idx]
    if abs(tmp - speed0[:,0][i]) <  omega*std:
      speed0_new[j].append(tmp)
      power0_new[j].append(power0[:,j].flat[idx])

colors = ['saddlebrown', 'springgreen', 'blueviolet', 'sandybrown', 'deeppink', 'darkviolet']

m, n = 0, 0
fig, axes = plt.subplots(4, 4, figsize=(25, 20), dpi=300, sharex=True, sharey=True)

for k in range(14):

  print("Current segment: %s; WT: %s"%(i, k))
  speed_list = np.array(speed0_new[k])
  speed_list = speed_list.reshape(len(speed_list), 1)
  power_list = np.array(power0_new[k])

  power_list = (power_list - 0)/2000 # set between 0 and 1
  power_list[power_list>1] = 1
  power_list[power_list<0] = 0

  model = BNN_gamma2()

  # Train
  model.load_weights("models4plot/density_%s.h5"%k)

  test_pred = model(speed_list)
  axes[m][n].scatter(speed_list, power_list , s=3, c='darkgray', label = 'Actual data')

  seed = 2
  # random.seed(seed)
  input_list = np.linspace(3, 13, 12)

  step = 0
  for p in range(0, len(input_list), 2):
    test_pred = model(np.array(input_list[p]).reshape(1,1,1))

    prediction_mean = test_pred.mean().numpy().tolist()[0]
    prediction_stdv = test_pred.stddev().numpy()[0]

    prediction_mean = np.array(prediction_mean)
    prediction_mean[prediction_mean<0] = 0
    prediction_mean[prediction_mean>1] = 1

    x = np.linspace(prediction_mean-2*prediction_stdv, prediction_mean+2*prediction_stdv, 200)

    res = norm.pdf(x, loc=prediction_mean, scale=prediction_stdv)
    res = (res - min(res))/(1.5*(max(res) - min(res)))

    axes[m][n].plot(input_list[p]+res, x , linewidth=1.5, color=colors[step], label='Speed = %sm/s'%round(input_list[p],2), linestyle='--')
    step += 1

  prediction = model(np.array(input_list).reshape(12 ,1,1))
  prediction_mean = prediction.mean().numpy()
  # prediction_mean = model(np.array(input_list).reshape(12 ,1,1)).numpy().flatten()
  prediction_mean[prediction_mean<0] = 0
  prediction_mean[prediction_mean>1] = 1
  axes[m][n].plot(input_list, prediction_mean, linewidth=2, c='#f12c60', label='Prediction mean')
  axes[m][n].set_xlim(2,13)
  axes[m][n].set_ylim(-0.05,1.05)

  axes[m][n].set_title('WT #%s'%(k+1), fontsize=15)

  if n==0:
    axes[m][n].set_ylabel('Wind Power', fontsize=15)

  if m==3:
    axes[m][n].set_xlabel('Wind Speed', fontsize=15)

  n += 1
  if n >3:
    m += 1
    n = 0

axes[3][2].set_xlabel('Wind Speed', fontsize=17)
axes[3][2].tick_params(labelsize=17)
axes[3][3].set_xlabel('Wind Speed', fontsize=17)
axes[3][3].tick_params(labelsize=17)

axes[0][0].legend(bbox_to_anchor=(0, 1.2), loc=3, borderaxespad=0.1, ncol=5, fontsize=20)

In [ ]:
prediction_mean

array([0.0103365 , 0.01505907, 0.0406231 , 0.09771577, 0.19132704,
       0.31426463, 0.44851053, 0.5754622 , 0.68436605, 0.7726282 ,
       0.84214896, 0.89626753], dtype=float32)